In [ ]:
#lrRNAseq GAST (Global Alignment Search Tool) visualiser
#run this notebook sequentially whilst filling in required variable paths and parameters
#written in all capital letters e.g. TRANSCRIPT_TO_GENOME_BLAST_PATH

In [ ]:
#must have the following libraries installed and imported: Pandas, Numpy, Matplotlib, regex, os, Biopython, ast, operator, warnings.
#must also have seqtk installed via bioconda i.e. conda install bioconda::seqtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import matplotlib.patches as patches
import matplotlib
from Bio import SeqIO
from matplotlib.ticker import MaxNLocator
from ast import literal_eval
from operator import itemgetter
import warnings
warnings.filterwarnings("ignore")
%run "./bioinformatic_functions.ipynb" #import functions from the "bioinformatic_functions" file.

In [ ]:
#Align long read RNAseq transcripts to a genomic region using BLAST with alignment parameters (-word_size, -evalue) of your choice.

#import lrRNAseq BLAST alignments to genomic region
TRANSCRIPT_TO_GENOME_BLAST_PATH = ""
#e.g. TRANSCRIPT_TO_GENOME_BLAST_PATH = "./lrRNAseq_to_Ef318A10.out"
all_blast = read_blast(TRANSCRIPT_TO_GENOME_BLAST_PATH)

In [ ]:
#calculate transcript filtration variables i.e. transcript coverage and maximal intron/alignment gap length
grouped_blast = all_blast.groupby("qseqid")

all_blast["Tr_coverage"] = 0
cov_dict = {}
dist_dict = {}
for tr, temp_df in grouped_blast:
    qlen = temp_df["qlen"].max()
    temp_df = temp_df.sort_values(by="qstart").reset_index(drop=True)
    q_overlaps, q_cov = calc_overlaps(temp_df, "q") 
    q_overlaps["Length"] = q_overlaps["Right"] - q_overlaps["Left"] 
    coverage = 100 / qlen * q_overlaps["Length"].sum()  
    cov_dict[tr] = coverage

    temp_df = temp_df.sort_values(by="sstart").reset_index(drop=True)
    q_overlaps, q_cov = calc_overlaps(temp_df, "s") 
    right = q_overlaps["Right"].tolist()
    left = q_overlaps["Left"].tolist()
    distances = []
    if len(right)>1:
        for i in range(len(right)-1):
            distances.append(left[i+1]-right[i])
    else:
        distances = [0]
    dist_dict[tr] = distances

# all_blast.loc[all_blast["qseqid"] == tr, "Tr_coverage"] = coverage
all_blast["Tr_coverage"] = all_blast["qseqid"].apply(lambda x: cov_dict[x])
all_blast["s_distances"] = all_blast["qseqid"].apply(lambda x: dist_dict[x])
all_blast["max_distance"] = all_blast["s_distances"].apply(lambda x: max(x))
all_blast["srange"] = all_blast.apply(lambda x: [x.sstart, x.send], axis = 1)
all_blast["qrange"] = all_blast.apply(lambda x: [x.qstart, x.qend], axis = 1)
all_blast["counts"] = all_blast.groupby("qseqid")["sseqid"].transform("count")

In [ ]:
cov50=all_blast.sort_values("sstart").reset_index(drop=True)

In [ ]:
#filter by maximal intron length
MAX_INTRON_LENGTH = 20000 #bp
cov50 = cov50[cov50["max_distance"]<MAX_INTRON_LENGTH]

In [ ]:
#filter by minimal percentage of transcript length covered by the genome as per collective BLAST alignments
MIN_COVERAGE = 25 #percent
cov50 = cov50[cov50["Tr_coverage"]>MIN_COVERAGE]

In [ ]:
#save filtered transcript ids to list in main directory
FILTERED_TRANSCRIPT_IDS = ""
#e.g. FILTERED_TRANCRIPT_IDS = "./aligned_lrRNAseq_cov25_filtered.txt"
write_list(cov50.drop_duplicates("qseqid")["qseqid"].tolist(), FILTERED_TRANCRIPT_IDS)

In [ ]:
#produce a FASTA file including only the transcripts that passed the filtration procedure
#NOTE: must have seqtk to run #conda install bioconda::seqtk
ALL_TRANSCRIPTS_PATH = "" # enter path to the bulk RNAseq file
#e.g. ALL_TRANSCRIPTS_PATH = "./all_transcripts.fasta"
FILTERED_TRANSCRIPTS_FASTA_PATH = "" #filtered transcripts FASTA path
#e.g. FILTERED_TRANSCRIPTS_FASTA_PATH = "./filtered_transcripts.fasta"
os.system(f"seqtk subseq {ALL_TRANSCRIPTS_PATH} {FILTERED_TRANSCRIPT_IDS} > {FILTERED_TRANSCRIPTS_FASTA_PATH})"

In [5]:
#import filtered transcripts FASTA and find ORF regions/stats
BARCODE_LEN = 0 #trim n barcode characters from both ends of transcripts #set to 39 if working with data provided in the example
cov50_fasta = parse_fasta(FILTERED_TRANSCRIPTS_FASTA_PATH,rm_barcode=BARCODE_LEN)[0]
all_stats = find_tr_ORF_stats(cov50_fasta["seq"].tolist(),cov50_fasta["id"].tolist(),tissue=True, include_seq=True)

0.251478910446167


In [ ]:
#import FASTA of the genomic region of interest e.g. BAC
GENOME_FASTA_PATH = ""
#e.g. GENOME_FASTA_PATH = "./Ef318A10.fst"
my_seq = "".join([line.strip() for line in open(GENOME_FASTA_PATH) if not line.startswith(">")])
seq_len = len(my_seq)

In [ ]:
#calculate ORF overlap statistics
all_s = all_stats[["id", "ORF_coords", "all_ORF_coords", "ORF_lengths", "num_ORFs_found"]]
all_s.rename(columns={"id":"qseqid"}, inplace=True)

cov50_ranges = cov50.groupby("qseqid").agg({"srange": "sum", "qrange": "sum"})
cov50_ranges = cov50_ranges.reset_index()

cov50_ranges["start"] = cov50_ranges["srange"].apply(lambda x: min(x))
cov50_ranges["end"] = cov50_ranges["srange"].apply(lambda x: max(x))

cov50_ranges["qstarts"] = cov50_ranges["qrange"].apply(lambda x: x[::2])
cov50_ranges["TrRange"] = cov50_ranges.apply(lambda x: [x["start"], x["end"]], axis=1)
cov50_ranges["TrLen"] = cov50_ranges["end"] - cov50_ranges["start"]

cov50_ranges = cov50_ranges.merge(right=all_s, on="qseqid",how="inner")
cov50_ranges["left_ORF_coords"] = cov50_ranges["all_ORF_coords"].apply(lambda x: [i[0] for i in x])
cov50_ranges["right_ORF_coords"] = cov50_ranges["all_ORF_coords"].apply(lambda x: [i[1] for i in x])

cov50_ranges["overlap_ORF_coords"] = cov50_ranges.apply(lambda x: calc_overlaps(pd.DataFrame({"Left":x.left_ORF_coords,"Right":x.right_ORF_coords}),by="LeftRight")[0]["Coords"].tolist(), axis=1)

cov50_ranges["qranges"] = cov50_ranges["qrange"].apply(lambda x: [[[x[::2], x[1::2]][0][i], [x[::2], x[1::2]][1][i]] for i in range(len([x[::2], x[1::2]][0]))])

cov50_ranges["in_ORF_matrix"] = cov50_ranges.apply(lambda x: [bool(list(pd.RangeIndex(i[0],i[1]).intersection(pd.RangeIndex(b[0],b[1])))) for i in x.qranges for b in x.overlap_ORF_coords], axis=1)
cov50_ranges["in_ORF_matrix_largest"] = cov50_ranges.apply(lambda x: [bool(list(pd.RangeIndex(i[0],i[1]).intersection(pd.RangeIndex(b[0],b[1])))) for i in x.qranges for b in [x.ORF_coords]], axis=1)

cov50_ranges["qranges_in_ORF"] = cov50_ranges.apply(lambda x: [any(z) for z in [[bool(list(pd.RangeIndex(i[0],i[1]).intersection(pd.RangeIndex(b[0],b[1])))) for i in x.qranges for b in x.overlap_ORF_coords][v:v+len(x.overlap_ORF_coords)] for v in range(0,len(x.in_ORF_matrix),len(x.overlap_ORF_coords))]], axis=1)
cov50_ranges["qranges_in_largest_ORF"] = cov50_ranges.apply(lambda x: [any(z) for z in [[bool(list(pd.RangeIndex(i[0],i[1]).intersection(pd.RangeIndex(b[0],b[1])))) for i in x.qranges for b in [x.ORF_coords]][v:v+len([x.ORF_coords])] for v in range(0,len(x.in_ORF_matrix_largest),len([x.ORF_coords]))]], axis=1)

cov50_ranges["qstarts_sorted_values"] = cov50_ranges["qstarts"].apply(lambda x: sort_with_order(x, make_equal=False)[0])
cov50_ranges["qstarts_sorted_order"] = cov50_ranges["qstarts"].apply(lambda x: sort_with_order(x, make_equal=False)[1])
cov50_ranges["qstarts_sorted_order_sequential"] = cov50_ranges["qstarts"].apply(lambda x: sort_with_order(x, make_sequential=True)[1])#true order of duplicate is lost

In [ ]:
#initialise plotting transcript arrangement
cov50_ranges_ = cov50_ranges.sort_values(by="start").reset_index(drop=True)
cov50_ranges_["TrOverlaps"] = ""
cov50_ranges_["TrSeparate"] = ""

for ii, ri in cov50_ranges_.iterrows():
    overlaps_with = []
    start = cov50_ranges_["start"][ii]
    end = cov50_ranges_["end"][ii]
    for ij, rj in cov50_ranges_.iterrows():
        if (end > cov50_ranges_["start"][ij] and end > cov50_ranges_["end"][ij] and start > cov50_ranges_["start"][ij] and start > cov50_ranges_["end"][ij]) or (end < cov50_ranges_["start"][ij] and end < cov50_ranges_["end"][ij] and start < cov50_ranges_["start"][ij] and start < cov50_ranges_["end"][ij]):
            overlaps_with.append(cov50_ranges_["qseqid"][ij])
    overlaps_with_ = [x for x in cov50_ranges_["qseqid"].tolist() if x not in overlaps_with]
    overlaps_with_ = [x for x in overlaps_with_ if x not in [cov50_ranges_["qseqid"][ii]]]
    cov50_ranges_["TrOverlaps"][ii] = overlaps_with_
    cov50_ranges_["TrSeparate"][ii] = overlaps_with

In [ ]:
cov50_ranges_["OverlapNum"] = cov50_ranges_["TrOverlaps"].apply(lambda x: len(x))
cov50_ranges_["SeparateNum"] = cov50_ranges_["TrSeparate"].apply(lambda x: len(x))

In [ ]:
cov50_ranges_bu = cov50_ranges_.copy()

In [ ]:
cov50_ranges_ =  cov50_ranges_bu.copy()
cov50_ranges_["NumExon"] = cov50_ranges_["srange"].apply(lambda x: len(x)/2)
cov50_ranges_["index"] = cov50_ranges_.index

In [ ]:
BATCH_SIZE = 40 #larger batches take exponentially longer to process, but produce more compact plot
lrRNAseq_plot_df = pd.DataFrame()
cov50_ranges_bu = pd.DataFrame()

ranges = []
for iii in range(0,cov50.drop_duplicates("qseqid").shape[0],BATCH_SIZE):
    ranges.append(iii)
ranges.append(cov50.drop_duplicates("qseqid").shape[0])

for iii in range(len(ranges)-1):
    print(iii)
    cov50_ranges_ = cov50_ranges.sort_values(by="start").reset_index(drop=True)[ranges[iii]:ranges[iii+1]]
    cov50_ranges_["TrOverlaps"] = ""
    cov50_ranges_["TrSeparate"] = ""
    
    for ii, ri in cov50_ranges_.iterrows():
        overlaps_with = []
        start = cov50_ranges_["start"][ii]
        end = cov50_ranges_["end"][ii]
        for ij, rj in cov50_ranges_.iterrows():
            if (end > cov50_ranges_["start"][ij] and end > cov50_ranges_["end"][ij] and start > cov50_ranges_["start"][ij] and start > cov50_ranges_["end"][ij]) or (end < cov50_ranges_["start"][ij] and end < cov50_ranges_["end"][ij] and start < cov50_ranges_["start"][ij] and start < cov50_ranges_["end"][ij]):
                overlaps_with.append(cov50_ranges_["qseqid"][ij])
        overlaps_with_ = [x for x in cov50_ranges_["qseqid"].tolist() if x not in overlaps_with]
        #overlaps_with_ = list(set(cov50_ranges_["qseqid"].tolist())-set(overlaps_with))
        overlaps_with_ = [x for x in overlaps_with_ if x not in [cov50_ranges_["qseqid"][ii]]]
        cov50_ranges_["TrOverlaps"][ii] = overlaps_with_
        cov50_ranges_["TrSeparate"][ii] = overlaps_with
    
    cov50_ranges_["OverlapNum"] = cov50_ranges_["TrOverlaps"].apply(lambda x: len(x))
    cov50_ranges_["SeparateNum"] = cov50_ranges_["TrSeparate"].apply(lambda x: len(x))
    
    if iii==0:
        cov50_ranges_bu = cov50_ranges_.copy()
    else:
        cov50_ranges_bu = pd.concat([cov50_ranges_bu, cov50_ranges_])
        cov50_ranges_bu = cov50_ranges_bu.reset_index(drop=True)
    
    cov50_ranges_["NumExon"] = cov50_ranges_["srange"].apply(lambda x: len(x)/2)
    cov50_ranges_["index"] = cov50_ranges_.index
    
    from operator import itemgetter
    
    def visit(TpM, chain, ii, chains,stop,chain_bu):
        for z in range(len(TpM)):        
            if stop == 0 and TpM[ii][z] == 1 and max(chain[:])<z:
                chain_bu = chain[:]
                chain.append(z)
                if sum(TpM[z]) > 0:
                    visit(TpM, chain, z, chains,stop,chain_bu)
                else:
                    chains.append(chain[:])
                    chain=chain_bu[:]
    
    
    #cov50_ranges_.sort_values(by="TrLen", ascending=False, inplace=True)
    cov50_ranges_.sort_values(by="TrLen", ascending=False)
    
    mut_cov50_ranges = cov50_ranges_.copy()
    TrsLeft = mut_cov50_ranges.index.tolist()
    
    final_coord_list = []
    final_coord_members = []
    final_coord_groups = []
    final_chain_members = []
    final_chain_ranges = []
    final_group_lengths = []
    saveTpM_list = []
    for idx, r in mut_cov50_ranges.iterrows():
        if idx in TrsLeft:
            TrPartners = mut_cov50_ranges[mut_cov50_ranges["index"] == idx]["TrSeparate"].values[0]
            #print(len(TrPartners))
            if len(TrPartners) > 0:
                TpM = [[0]*len(TrPartners) for _ in range(len(TrPartners))]
                for trp in TrPartners:
                    #print(trp)
                    TpPartners = mut_cov50_ranges[mut_cov50_ranges["qseqid"]==trp]["TrSeparate"].values[0]
                    TrTpPartners = [x for x in TrPartners if x in TpPartners]
                    # print(mut_cov50_ranges["qseqid"][idx])
                    # print("idx", idx)
                    # print(TrPartners)
                    # print(TpPartners)
                    # print(len(TpPartners))
                    # print(TrTpPartners)
                    # print(len(TrTpPartners))
                    for trtpp in TrTpPartners:
                        if TrPartners.index(trp)<TrPartners.index(trtpp):
                            TpM[TrPartners.index(trp)][TrPartners.index(trtpp)] = 1
    
                if len(TpM)>1:
                    saveTpM_list.append(TpM)
                if sum([x for i in TpM for x in i]) == 0:
                    
                    total_lengths = []
                    
                    for tr in TrPartners:
                        total_length = mut_cov50_ranges[mut_cov50_ranges["qseqid"] == tr]["TrLen"].tolist()[0]
                        total_lengths.append(total_length)
                    
                    index_max = max(range(len(total_lengths)), key=total_lengths.__getitem__)
                    
                    best_trs_df = pd.concat([mut_cov50_ranges[mut_cov50_ranges["index"] == idx], mut_cov50_ranges[mut_cov50_ranges["qseqid"] == TrPartners[index_max]]])
                    best_trs_df = best_trs_df.sort_values(by="start")
                    coord_list = best_trs_df["srange"].tolist()
                    coord_list_ = [x for l in coord_list for x in l]
                    coord_members = len(coord_list_)/2
                    coord_groups = best_trs_df["NumExon"].tolist()
                    
                    final_coord_list.append(coord_list_)
                    final_coord_members.append(coord_members)
                    final_chain_members.append(best_trs_df["qseqid"].tolist())
                    final_coord_groups.append(coord_groups)
                    final_chain_ranges.append(best_trs_df["TrRange"].tolist())
                    final_group_lengths.append(best_trs_df["TrLen"].sum())
                    
                    trs_to_remove = best_trs_df.index.tolist()
                    TrsLeft = [x for x in TrsLeft if x not in trs_to_remove]
                    
                    cov50_ranges_.drop(trs_to_remove, inplace=True)
                    
                    for ii, ri in cov50_ranges_.iterrows():
                        overlaps_with = []
                        start = cov50_ranges_["start"][ii]
                        end = cov50_ranges_["end"][ii]
                        for ij, rj in cov50_ranges_.iterrows():
                            if (end > cov50_ranges_["start"][ij] and end > cov50_ranges_["end"][ij] and start > cov50_ranges_["start"][ij] and start > cov50_ranges_["end"][ij]) or (end < cov50_ranges_["start"][ij] and end < cov50_ranges_["end"][ij] and start < cov50_ranges_["start"][ij] and start < cov50_ranges_["end"][ij]):
                                overlaps_with.append(cov50_ranges_["qseqid"][ij])
                        overlaps_with_ = [x for x in cov50_ranges_["qseqid"].tolist() if x not in overlaps_with]
                        #overlaps_with_ = list(set(cov50_ranges_["qseqid"].tolist())-set(overlaps_with))
                        overlaps_with_ = [x for x in overlaps_with_ if x not in [cov50_ranges_["qseqid"][ii]]]
                        cov50_ranges_["TrOverlaps"][ii] = overlaps_with_
                        cov50_ranges_["TrSeparate"][ii] = overlaps_with
                    
                    cov50_ranges_["OverlapNum"] = cov50_ranges_["TrOverlaps"].apply(lambda x: len(x))
                    cov50_ranges_["SeparateNum"] = cov50_ranges_["TrSeparate"].apply(lambda x: len(x))
                
                    mut_cov50_ranges = cov50_ranges_.copy()
                    
                else:
                    chains = []
                    stop = 0
                    for i in range(len(TpM)):
                        for j in range(len(TpM)):
                            if TpM[i][j] == 1 and stop==0:
                                chain = [i,j] #0,7
                                chain_bkup = chain[:]
                                chains.append(chain[:])
                                if sum(TpM[j]) > 0:
                                    visit(TpM, chain, j, chains, stop, chain_bkup)
                    
                    correction = []
                    for elem in chains:
                        if sum(TpM[elem[-1]])==0 and sum(TpM[elem[-2]])==0:
                            correction.append(elem)
                    
                    chains = [x for x in chains if x not in correction]
                    new_chains = []
                    for elem in chains:
                        if elem not in new_chains:
                            new_chains.append(elem)
                    chains = new_chains
                   # print(chains)
                    total_lengths = []
                    
                    for chain in chains:
                        total_length = 0
                        for i in chain:
                            total_length += mut_cov50_ranges[mut_cov50_ranges["qseqid"] == TrPartners[i]]["TrLen"].tolist()[0]
                        total_lengths.append(total_length)
                    #print(total_lengths)
                    index_max = max(range(len(total_lengths)), key=total_lengths.__getitem__)
                
                    best_chain = chains[index_max]
                
                    best_trs = itemgetter(*best_chain)(TrPartners)
                    #print("besttrs1", best_trs)
                    #print("idx", idx)
                    best_trs_df = pd.concat([mut_cov50_ranges[mut_cov50_ranges["index"] == idx], mut_cov50_ranges[mut_cov50_ranges["qseqid"].isin(best_trs)]])
                    #print("besttrs2", best_trs_df)
                    best_trs_df = best_trs_df.sort_values(by="start")
                    coord_list = best_trs_df["srange"].tolist()
                    coord_list_ = [x for l in coord_list for x in l]
                    coord_members = len(coord_list_)/2
                    coord_groups = best_trs_df["NumExon"].tolist()
                    
                    final_coord_list.append(coord_list_)
                    final_coord_members.append(coord_members)
                    final_chain_members.append(best_trs_df["qseqid"].tolist())
                    final_coord_groups.append(coord_groups)
                    final_chain_ranges.append(best_trs_df["TrRange"].tolist())
                    final_group_lengths.append(best_trs_df["TrLen"].sum())
                
                    trs_to_remove = best_trs_df.index.tolist()
                    TrsLeft = [x for x in TrsLeft if x not in trs_to_remove]
                    
                    cov50_ranges_.drop(trs_to_remove, inplace=True)
                    
                    for ii, ri in cov50_ranges_.iterrows():
                        overlaps_with = []
                        start = cov50_ranges_["start"][ii]
                        end = cov50_ranges_["end"][ii]
                        for ij, rj in cov50_ranges_.iterrows():
                            if (end > cov50_ranges_["start"][ij] and end > cov50_ranges_["end"][ij] and start > cov50_ranges_["start"][ij] and start > cov50_ranges_["end"][ij]) or (end < cov50_ranges_["start"][ij] and end < cov50_ranges_["end"][ij] and start < cov50_ranges_["start"][ij] and start < cov50_ranges_["end"][ij]):
                                overlaps_with.append(cov50_ranges_["qseqid"][ij])
                        overlaps_with_ = [x for x in cov50_ranges_["qseqid"].tolist() if x not in overlaps_with]
                        #overlaps_with_ = list(set(cov50_ranges_["qseqid"].tolist())-set(overlaps_with))
                        overlaps_with_ = [x for x in overlaps_with_ if x not in [cov50_ranges_["qseqid"][ii]]]
                        cov50_ranges_["TrOverlaps"][ii] = overlaps_with_
                        cov50_ranges_["TrSeparate"][ii] = overlaps_with
                    
                    cov50_ranges_["OverlapNum"] = cov50_ranges_["TrOverlaps"].apply(lambda x: len(x))
                    cov50_ranges_["SeparateNum"] = cov50_ranges_["TrSeparate"].apply(lambda x: len(x))
                
                    mut_cov50_ranges = cov50_ranges_.copy()
            else:
                coord_list_ = mut_cov50_ranges[mut_cov50_ranges["index"] == idx]["srange"].values[0]
                coord_members = len(coord_list_)/2            
                coord_groups = mut_cov50_ranges[mut_cov50_ranges["index"] == idx]["NumExon"].values[0]
                
                final_coord_list.append(coord_list_)
                final_coord_members.append(coord_members)
                final_chain_members.append(mut_cov50_ranges[mut_cov50_ranges["index"] == idx]["qseqid"].values[0])
                final_coord_groups.append(coord_groups)
                final_chain_ranges.append(mut_cov50_ranges[mut_cov50_ranges["index"] == idx]["TrRange"].values[0])
                final_group_lengths.append(mut_cov50_ranges[mut_cov50_ranges["index"] == idx]["TrLen"].values[0])
                
                trs_to_remove = [idx]
                TrsLeft = [x for x in TrsLeft if x not in trs_to_remove]
                
                cov50_ranges_.drop(trs_to_remove, inplace=True)
                
                for ii, ri in cov50_ranges_.iterrows():
                    overlaps_with = []
                    start = cov50_ranges_["start"][ii]
                    end = cov50_ranges_["end"][ii]
                    for ij, rj in cov50_ranges_.iterrows():
                        if (end > cov50_ranges_["start"][ij] and end > cov50_ranges_["end"][ij] and start > cov50_ranges_["start"][ij] and start > cov50_ranges_["end"][ij]) or (end < cov50_ranges_["start"][ij] and end < cov50_ranges_["end"][ij] and start < cov50_ranges_["start"][ij] and start < cov50_ranges_["end"][ij]):
                            overlaps_with.append(cov50_ranges_["qseqid"][ij])
                    overlaps_with_ = [x for x in cov50_ranges_["qseqid"].tolist() if x not in overlaps_with]
                    #overlaps_with_ = list(set(cov50_ranges_["qseqid"].tolist())-set(overlaps_with))
                    overlaps_with_ = [x for x in overlaps_with_ if x not in [cov50_ranges_["qseqid"][ii]]]
                    cov50_ranges_["TrOverlaps"][ii] = overlaps_with_
                    cov50_ranges_["TrSeparate"][ii] = overlaps_with
                
                cov50_ranges_["OverlapNum"] = cov50_ranges_["TrOverlaps"].apply(lambda x: len(x))
                cov50_ranges_["SeparateNum"] = cov50_ranges_["TrSeparate"].apply(lambda x: len(x))
            
                mut_cov50_ranges = cov50_ranges_.copy()
    
    for idx, row in mut_cov50_ranges.iterrows():
        coord_list_ = mut_cov50_ranges["srange"][idx]
        coord_members = len(coord_list_)/2            
        coord_groups = mut_cov50_ranges["NumExon"][idx]
        
        final_coord_list.append(coord_list_)
        final_coord_members.append(coord_members)
        final_chain_members.append(mut_cov50_ranges["qseqid"][idx])
        final_coord_groups.append(coord_groups)
        final_chain_ranges.append(mut_cov50_ranges["TrRange"][idx])
        final_group_lengths.append(mut_cov50_ranges["TrLen"][idx])
    
    lrRNAseq_plot_df_pre = pd.DataFrame()
    lrRNAseq_plot_df_pre["coords"] = final_coord_list
    lrRNAseq_plot_df_pre["total_exons"] = final_coord_members
    lrRNAseq_plot_df_pre["TrNames"] = final_chain_members
    lrRNAseq_plot_df_pre["exons_per_tr"] = final_coord_groups
    lrRNAseq_plot_df_pre["range_per_tr"] = final_chain_ranges
    lrRNAseq_plot_df_pre["Total_aln_len"] = final_group_lengths
    lrRNAseq_plot_df = pd.concat([lrRNAseq_plot_df, lrRNAseq_plot_df_pre])
    lrRNAseq_plot_df = lrRNAseq_plot_df.sort_values("Total_aln_len",ascending=False)
    lrRNAseq_plot_df = lrRNAseq_plot_df.reset_index(drop=True)

In [ ]:
cov50_ranges_ =  cov50_ranges_bu.copy()
cov50_ranges_["NumExon"] = cov50_ranges_["srange"].apply(lambda x: len(x)/2)
cov50_ranges_["index"] = cov50_ranges_.index

In [ ]:
#combine transcript plot dataframe with the filtered BLAST dataframe
lrRNAseq_plot_df["ORF_coords"] = ""
lrRNAseq_plot_df["all_ORF_coords"] = ""
lrRNAseq_plot_df["qranges_in_largest_ORF"] = ""
lrRNAseq_plot_df["qranges_in_ORF"] = ""
lrRNAseq_plot_df["qstarts_sorted_values"] = ""
lrRNAseq_plot_df["qstarts_sorted_order"] = ""
lrRNAseq_plot_df["qstarts_sorted_order_sequential"] = ""
lrRNAseq_plot_df["qstarts"] = ""
lrRNAseq_plot_df["index"] = lrRNAseq_plot_df.index

for idx, row in lrRNAseq_plot_df.iterrows():
    qstarts_sorted_order_sequential_list = []
    qstarts_sorted_order_list = []
    qstarts_sorted_values_list = []
    qranges_in_largest_ORF_list = []
    qranges_in_ORF_list = []
    all_ORF_coords_list = []
    ORF_coords_list = []
    qstarts_list = []
    if type(lrRNAseq_plot_df["TrNames"][idx])==str:
        id = lrRNAseq_plot_df["TrNames"][idx]
        qstarts_sorted_order_sequential_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts_sorted_order_sequential"].tolist()[0])
        qstarts_sorted_order_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts_sorted_order"].tolist()[0])
        qstarts_sorted_values_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts_sorted_values"].tolist()[0])
        qranges_in_largest_ORF_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qranges_in_largest_ORF"].tolist()[0])
        qranges_in_ORF_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qranges_in_ORF"].tolist()[0])
        all_ORF_coords_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["all_ORF_coords"].tolist()[0])
        ORF_coords_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["ORF_coords"].tolist()[0])
        qstarts_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts"].tolist()[0])
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "ORF_coords"] = str(ORF_coords_list)
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "all_ORF_coords"] = str(all_ORF_coords_list)
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qranges_in_largest_ORF"] = str(qranges_in_largest_ORF_list)
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qranges_in_ORF"] = str(qranges_in_ORF_list)
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts_sorted_values"] = str(qstarts_sorted_values_list)
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts_sorted_order"] = str(qstarts_sorted_order_list)
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts_sorted_order_sequential"] = str(qstarts_sorted_order_sequential_list)
        lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts"] = str(qstarts_list)
    else:    
        for id in lrRNAseq_plot_df["TrNames"][idx]:
            qstarts_sorted_order_sequential_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts_sorted_order_sequential"].tolist()[0])
            qstarts_sorted_order_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts_sorted_order"].tolist()[0])
            qstarts_sorted_values_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts_sorted_values"].tolist()[0])
            qranges_in_largest_ORF_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qranges_in_largest_ORF"].tolist()[0])
            qranges_in_ORF_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qranges_in_ORF"].tolist()[0])
            all_ORF_coords_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["all_ORF_coords"].tolist()[0])
            ORF_coords_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["ORF_coords"].tolist()[0])
            qstarts_list.append(cov50_ranges_[cov50_ranges_["qseqid"]==id]["qstarts"].tolist()[0])
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "ORF_coords"] = str(ORF_coords_list)
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "all_ORF_coords"] = str(all_ORF_coords_list)
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qranges_in_largest_ORF"] = str(qranges_in_largest_ORF_list)
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qranges_in_ORF"] = str(qranges_in_ORF_list)
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts_sorted_values"] = str(qstarts_sorted_values_list)
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts_sorted_order"] = str(qstarts_sorted_order_list)
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts_sorted_order_sequential"] = str(qstarts_sorted_order_sequential_list)
            lrRNAseq_plot_df.loc[lrRNAseq_plot_df["index"]== idx, "qstarts"] = str(qstarts_list)

lrRNAseq_plot_df["ORF_coords"] = lrRNAseq_plot_df["ORF_coords"].apply(lambda x: eval(x))
lrRNAseq_plot_df["all_ORF_coords"] = lrRNAseq_plot_df["all_ORF_coords"].apply(lambda x: eval(x))
lrRNAseq_plot_df["qranges_in_largest_ORF"] = lrRNAseq_plot_df["qranges_in_largest_ORF"].apply(lambda x: eval(x))
lrRNAseq_plot_df["qranges_in_ORF"] = lrRNAseq_plot_df["qranges_in_ORF"].apply(lambda x: eval(x))
lrRNAseq_plot_df["qstarts_sorted_values"] = lrRNAseq_plot_df["qstarts_sorted_values"].apply(lambda x: eval(x))
lrRNAseq_plot_df["qstarts_sorted_order"] = lrRNAseq_plot_df["qstarts_sorted_order"].apply(lambda x: eval(x))
lrRNAseq_plot_df["qstarts_sorted_order_sequential"] = lrRNAseq_plot_df["qstarts_sorted_order_sequential"].apply(lambda x: eval(x))
lrRNAseq_plot_df["qstarts"] = lrRNAseq_plot_df["qstarts"].apply(lambda x: eval(x))

In [ ]:
#plotting requirements
lrRNAseq_plot_df = lrRNAseq_plot_df.sort_values(by="Total_aln_len", ascending=False)
lrRNAseq_plot_df["coverage"] = lrRNAseq_plot_df["total_exons"].apply(lambda x: [0]+[0,1,1,0]*int(x)+[0] )

In [ ]:
#further processing of plotting coordinates
lrRNAseq_plot_df["coords2"] = ""
for idx, row in lrRNAseq_plot_df.iterrows():
    coords = lrRNAseq_plot_df["coords"][idx]
    coords1 = coords[::2]
    coords1_add = [x - 1 for x in coords1]
    coords2 = coords[1::2]
    coords2_add = [x + 1 for x in coords2]
    lrRNAseq_plot_df["coords2"][idx] = lrRNAseq_plot_df["coords"][idx] + coords1_add + coords2_add 
lrRNAseq_plot_df["coords2"] = lrRNAseq_plot_df["coords2"].apply(lambda x: [0]+x+[len(my_seq)])

In [ ]:
#plotting coordinate statistics
lrRNAseq_plot_df["min"] = lrRNAseq_plot_df["coords"].apply(lambda x: min(x))
lrRNAseq_plot_df["max"] = lrRNAseq_plot_df["coords"].apply(lambda x: max(x))
lrRNAseq_plot_df["span"] = lrRNAseq_plot_df["max"] - lrRNAseq_plot_df["min"]
lrRNAseq_plot_df.sort_values(by="span", ascending=False, inplace=True)

In [ ]:
#translate order and get coord ranges
lrRNAseq_plot_df["qstarts_sorted_order_translated"] = lrRNAseq_plot_df["qstarts_sorted_order_sequential"].apply(lambda x: [translate_order(i) for i in x])
lrRNAseq_plot_df["coord_ranges"] = lrRNAseq_plot_df["coords"].apply(lambda x: [[[x[::2],x[1::2]][0][i],[x[::2],x[1::2]][1][i]]for i in range(len([x[::2],x[1::2]][0]))])

In [ ]:
#create order dictionary for each transcript
dict_list_outer = []
for idx, row in lrRNAseq_plot_df.iterrows():
    dict_list = []
    for i in range(len(lrRNAseq_plot_df["qstarts_sorted_order_sequential"][idx])):        
        d = {}
        for j in range(len(lrRNAseq_plot_df["qstarts_sorted_order_sequential"][idx][i])):
            d[lrRNAseq_plot_df["qstarts_sorted_values"][idx][i][j] ] = lrRNAseq_plot_df["qstarts_sorted_order_sequential"][idx][i][j]
        dict_list.append(d)
    dict_list_outer.append(dict_list)
lrRNAseq_plot_df["order_dict"] = dict_list_outer

In [ ]:
#use order dictionary to check order of alignment regions in transcripts
ordered_list_outer = []
for idx, row in lrRNAseq_plot_df.iterrows():
    ordered_list = []
    for j in range(len(lrRNAseq_plot_df["order_dict"][idx])):
        
        o_dict = lrRNAseq_plot_df["order_dict"][idx][j]
        qstarts = lrRNAseq_plot_df["qstarts"][idx][j]
        ordered = []
        
        for i in range(len(qstarts)):
            ordered.append(o_dict[qstarts[i]])
        ordered_list.append(ordered)
    ordered_list_outer.append(ordered_list)
lrRNAseq_plot_df["exon_synteny"] = ordered_list_outer
lrRNAseq_plot_df["centre_coords"] = lrRNAseq_plot_df["coord_ranges"].apply(lambda x: [int(i[0]+((i[1]-i[0])/2)) for i in x])

In [ ]:
#ordered exon indexes and correction for closely bundled centre coordinates for better visibility of the numbers in the plot
lrRNAseq_plot_df["syntenous_indexes"] = lrRNAseq_plot_df["exon_synteny"].apply(lambda x: [syntenous_indexes(i) for i in x])
lrRNAseq_plot_df["centre_coords_corrected"] = lrRNAseq_plot_df["centre_coords"].apply(lambda x: space_elements(x))

In [ ]:
#required for plotting multi-transcript syntentic regions
#convert exons_per_tr to lists
exons_per_tr_list = []
for idx, row in lrRNAseq_plot_df.iterrows():
    ex_per_tr = lrRNAseq_plot_df["exons_per_tr"][idx]
    if type(ex_per_tr) == float:
        exons_per_tr_list.append([int(ex_per_tr)])
    else:
        exons_per_tr_list.append(ex_per_tr)
lrRNAseq_plot_df["exons_per_tr"] = exons_per_tr_list


In [ ]:
#make syntenous_indexes lists have even number of members
new_xs = []
for idx, row in lrRNAseq_plot_df.iterrows():
    x = lrRNAseq_plot_df["syntenous_indexes"][idx].copy()
# x = [[0,3,0],[0],[0,4,5,7]]
    new_x = []
    for i in x:
        if len(i)%2!=0:
            i.append(i[len(i)-1])
        new_x.append(i)
    new_xs.append(new_x)
lrRNAseq_plot_df["syntenous_indexes_even"] = new_xs

In [ ]:
#increment subsequent transcript lists by number of exons in previous transcripts
syn_idxs_corr_list = []
for idx, row in lrRNAseq_plot_df.iterrows():
    syn_idxs = lrRNAseq_plot_df["syntenous_indexes_even"][idx].copy()
    # syn_idxs = [[0, 1], [0, 5, 6, 7, 8, 14, 15, 16, 22],[0,3]]
    ex_per_tr = lrRNAseq_plot_df["exons_per_tr"][idx].copy()
    # ex_per_tr = [2.0, 23.0, 4]
    num_trs = len(syn_idxs)
    increment = 0
    syn_idxs_corrected = []
    syn_idxs_corrected.append(syn_idxs[0])
    
    for i in range(1,num_trs):
        increment += ex_per_tr[i-1]
        syn_idxs_corrected.append((np.array(syn_idxs[i])+increment).tolist())
    syn_idxs_corr_list.append(syn_idxs_corrected)
lrRNAseq_plot_df["syn_idxs_corrected"] = syn_idxs_corr_list

In [ ]:
#finalising information
lrRNAseq_plot_df["qranges_in_ORF_unwrapped"] = lrRNAseq_plot_df["qranges_in_ORF"].apply(lambda x: [i for i in extract_nested_values(x)] )
lrRNAseq_plot_df["qranges_in_largest_ORF_unwrapped"] = lrRNAseq_plot_df["qranges_in_largest_ORF"].apply(lambda x: [i for i in extract_nested_values(x)] )
lrRNAseq_plot_df["exon_synteny_unwrapped"] = lrRNAseq_plot_df["exon_synteny"].apply(lambda x: [i for i in extract_nested_values(x)] )
lrRNAseq_plot_df["syntenous_indexes_unwrapped"] = lrRNAseq_plot_df["syn_idxs_corrected"].apply(lambda x: [i for i in extract_nested_values(x)] )
lrRNAseq_plot_df["syntenous_indexes_int_unwrapped"]  = lrRNAseq_plot_df["syntenous_indexes_unwrapped"].apply(lambda x: [int(i) for i in x])

In [ ]:
#plot results
seq_length = seq_len
matplotlib.rcParams["agg.path.chunksize"] = 10000
plt.rcParams['agg.path.chunksize'] = 10000
figsize_param_w = min(int(seq_length/1100), 50)
figsize_param_h = int(lrRNAseq_plot_df.shape[0]/2.5)
plt.rcParams['figure.figsize'] = [figsize_param_w, figsize_param_h]
plt.rcParams['figure.dpi'] = 140

nrows = lrRNAseq_plot_df.shape[0]
fig, axes = plt.subplots(nrows=nrows, ncols=1, sharex=False)
plt.subplots_adjust(top=0.9, bottom=0.1, hspace=0)
axes[lrRNAseq_plot_df.shape[0]-1].set_xlabel("")


lrRNAseq_plot_df.reset_index(drop=True, inplace=True)

for idx, row in lrRNAseq_plot_df.iterrows():
#plot all alignment regions
    temp_plot_df = pd.DataFrame()
    temp_plot_df["coords"] = lrRNAseq_plot_df["coords2"][idx]
    temp_plot_df.sort_values(by="coords", inplace=True)
    temp_plot_df["coverage"] = lrRNAseq_plot_df["coverage"][idx]
    temp_plot_df.plot.area(x="coords", y= "coverage", figsize=(figsize_param_w, figsize_param_h), grid=True,
                           legend=False, ax = axes[idx], color='r', linewidth=1, xlim=[0,seq_length],zorder=1)
#plot line if only 1 line in lane
    temp_lines = pd.DataFrame()
    if type(lrRNAseq_plot_df["range_per_tr"][idx][0]) == int:
        temp_lines["coord"] = lrRNAseq_plot_df["range_per_tr"][idx]
        temp_lines["midline"] = 0.5
        temp_lines.plot.line(x="coord", y= "midline", figsize=(figsize_param_w, figsize_param_h), grid=True, 
                             legend=False, ax = axes[idx], color='b', linewidth=3, xlim=[0,seq_length])
#plot line if multiple lines in lane
    else:
        c = ["b","g","m","c","w","grey","k", "silver", "beige","teal","lightskyblue","r","y","lightgreen","peru"]
        i=0
        for line in lrRNAseq_plot_df["range_per_tr"][idx]:
            temp_lines["coord"] = line
            temp_lines["midline"] = 0.5
            temp_lines.plot.line(x="coord", y= "midline", figsize=(figsize_param_w, figsize_param_h), grid=True,
                                   legend=False, ax = axes[idx], color=c[i], linewidth=4, xlim=[0,seq_length])
            i+=1
#plot alignment region in ORF as a rectangle patch - border only
    v = 0
    coord_ranges = lrRNAseq_plot_df["coord_ranges"][idx]
    qranges_in_ORF = lrRNAseq_plot_df["qranges_in_ORF_unwrapped"][idx]
    # qranges_in_ORF = lrRNAseq_plot_df["qranges_in_ORF"][idx]
    for c in coord_ranges:
        if qranges_in_ORF[v] == True:
            a2 = patches.Rectangle((c[0],0),c[1]-c[0],1,color="gold", fill=None,zorder=1)
            axes[idx].add_patch(a2)
        v+=1
#plot alignment region in largest ORF as a rectangle patch - border and fill
    v = 0
    qranges_in_largest_ORF = lrRNAseq_plot_df["qranges_in_largest_ORF_unwrapped"][idx]
    for c in coord_ranges:
        if qranges_in_largest_ORF[v] == True:
            a1 = patches.Rectangle((c[0],0),c[1]-c[0],1,color="gold",zorder=99)
            a2 = patches.Rectangle((c[0],0),c[1]-c[0],1,color="gold", fill=None,zorder=1)
            axes[idx].add_patch(a1)
            axes[idx].add_patch(a2)
        v+=1

#plot text of syntentic order 
    centre_coords = lrRNAseq_plot_df["centre_coords"][idx]
    syntentic_order = lrRNAseq_plot_df["exon_synteny_unwrapped"][idx]
    for i in range(len(centre_coords)):
        fig.text(centre_coords[i], int((nrows-idx)/(nrows+1))+0.5, f"{syntentic_order[i]}", horizontalalignment="center", transform=axes[idx].transData, verticalalignment="center", fontsize=15, color="black")

#plot syntentic frame
    v = 0
    coord_ranges = lrRNAseq_plot_df["coord_ranges"][idx]
    structure = lrRNAseq_plot_df["syntenous_indexes_int_unwrapped"][idx]
    syntentic_coords = []
    for i in range(0,len(structure)-1,2):
        start = coord_ranges[structure[i]][0]
        end = coord_ranges[structure[i+1]][1]
        syntentic_coords.append([start,end])
    for c in syntentic_coords:
        # a4 = patches.Rectangle((c[0],0)c[1]-c[0],1,color="pink",zorder=1)
        a3 = patches.Rectangle((c[0],0),c[1]-c[0],1,color="pink",zorder=0)
        axes[idx].add_patch(a3)
        # axes[idx].add_patch(a4)
        v+=1

for idx in range(nrows):
    if idx != 0 and idx != nrows-1:
        axes[idx].set_xticklabels([])
        axes[idx].set_xlabel("")
        axes[idx].minorticks_on()
        
        # axes[idx].tick_params(axis='both', which='major', labelsize=30, pad=15, width=3, size=8) #labelbottom=True,labeltop=True,bottom=True,top=True
        # axes[idx].tick_params(axis='both', which='minor', labelsize=30, pad=15, width=2, size=5) #labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='y', which='major', labelsize=30, pad=0, width=0, size=0) #labelbottom=True,labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='y', which='minor', labelsize=0, pad=0, width=0, size=0) #labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='x', which='major', labelsize=30, pad=15, width=3, size=8,labelbottom=True,labeltop=True,bottom=True,top=True) #labelbottom=True,labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='x', which='minor', labelsize=30, pad=15, width=2, size=5,labelbottom=True,labeltop=True,bottom=True,top=True) #labeltop=True,bottom=True,top=True
        axes[idx].grid(which="major", axis="x", color = 'black', linestyle = '--', dashes=(5, 5), linewidth = 1.35, alpha=0.65) 
        axes[idx].grid(which="minor", axis="x", color = 'black', linestyle = '-', dashes=(4, 5), linewidth = 0.85, alpha=0.45)    
        for axis in ["top", "bottom", "left", "right"]: 
            axes[idx].spines[axis].set_linewidth(2.5)
        axes[0].xaxis.tick_top()
        
    elif idx == 0:
        axes[idx].set_xlabel("")
        axes[idx].minorticks_on()
        
        # axes[idx].tick_params(axis='both', which='major', labelsize=30, pad=15, width=3, size=8) #labelbottom=True,labeltop=True,bottom=True,top=True
        # axes[idx].tick_params(axis='both', which='minor', labelsize=30, pad=15, width=2, size=5) #labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='y', which='major', labelsize=30, pad=0, width=0, size=0) #labelbottom=True,labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='y', which='minor', labelsize=0, pad=0, width=0, size=0) #labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='x', which='major', labelsize=30, pad=15, width=3, size=8,labelbottom=0,labeltop=True,bottom=0,top=True) #labelbottom=True,labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='x', which='minor', labelsize=30, pad=15, width=2, size=5,labelbottom=0,labeltop=True,bottom=0,top=True) #labeltop=True,bottom=True,top=True
        axes[idx].grid(which="major", axis="x", color = 'black', linestyle = '--', dashes=(5, 5), linewidth = 1.35, alpha=0.65) 
        axes[idx].grid(which="minor", axis="x", color = 'black', linestyle = '-', dashes=(4, 5), linewidth = 0.85, alpha=0.45)    
        for axis in ["top", "bottom", "left", "right"]: 
            axes[idx].spines[axis].set_linewidth(2.5)
        axes[0].xaxis.tick_top()

    elif idx == nrows-1:
        # axes[idx].tick_params(axis='both', which='major', labelsize=30, pad=15, width=3, size=8) #labelbottom=True,labeltop=True,bottom=True,top=True
        # axes[idx].tick_params(axis='both', which='minor', labelsize=30, pad=15, width=2, size=5) #labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='y', which='major', labelsize=30, pad=0, width=0, size=0) #labelbottom=True,labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='y', which='minor', labelsize=0, pad=0, width=0, size=0) #labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='x', which='major', labelsize=30, pad=15, width=3, size=8,labelbottom=True,labeltop=0,bottom=True,top=0) #labelbottom=True,labeltop=True,bottom=True,top=True
        axes[idx].tick_params(axis='x', which='minor', labelsize=30, pad=15, width=2, size=5,labelbottom=True,labeltop=0,bottom=True,top=0) #labeltop=True,bottom=True,top=True
        axes[idx].grid(which="major", axis="x", color = 'black', linestyle = '--', dashes=(5, 5), linewidth = 1.35, alpha=0.65) 
        axes[idx].grid(which="minor", axis="x", color = 'black', linestyle = '-', dashes=(4, 5), linewidth = 0.85, alpha=0.45)    
        for axis in ["top", "bottom", "left", "right"]: 
            axes[idx].spines[axis].set_linewidth(2.5)
        axes[0].xaxis.tick_top()
        axes[idx].set_xlabel("")
        axes[idx].minorticks_on()
        
    for z, label in enumerate(axes[idx].get_yticklabels()):
        if z >= 0 and z < len(axes[idx].get_yticklabels()):
            label.set_visible(False)
    
    axes[idx].set_yticklabels([])

In [ ]:
#the rows of plot are indexed by the index of lrRNAseq_plot_df dataframe
#TrNames column of lrRNAseq_plot_df contains the names of transcript for the corresponding row of the plot